In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import os

# Folder za spremanje confusion matrica
os.makedirs('confusion_matrices', exist_ok=True)

# Definicije datoteka
train_files = ['train-1.csv', 'train-2.csv', 'train-3.csv']
test_files = ['test-1.csv', 'test-2.csv', 'test-3.csv']
test_names = ['Test 1', 'Test 2', 'Test 3']

text_column = 'Sentence'
target_column = 'Label'

# Funkcija za učitavanje podataka
def load_data(file):
    df = pd.read_csv(file)
    X_text = df[text_column].astype(str)
    y = df[target_column]
    return X_text, y

# Funkcija za učitavanje i spajanje više train setova
def load_data_combined(files):
    X_all = []
    y_all = []
    for file in files:
        X_text, y = load_data(file)
        X_all.extend(X_text)
        y_all.extend(y)
    return X_all, y_all

# Ažurirani modeli
models = [
    ('1.a', 'Machine learning', 'Logistic regression', LogisticRegression(max_iter=5000, solver='liblinear', class_weight='balanced')),
    ('1.b', 'Machine learning', 'SVM RBF kernel', SVC(class_weight='balanced', kernel='rbf', random_state=42))
]

# Priprema tablice za rezultate
table = []

# Učitavanje kombiniranih trening podataka
X_train_text, y_train = load_data_combined(train_files)

# TF-IDF vektorizacija s proširenim parametrima
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_train = vectorizer.fit_transform(X_train_text)

# Treniranje i evaluacija
for code, method, algorithm, model in models:
    print(f"\nTreniranje modela: {algorithm}...")
    model.fit(X_train, y_train)
    
    row_train = [f"{code}.i", method, algorithm, "Train combined"]
    for idx, test_file in enumerate(test_files):
        print(f"Predikcija na {test_names[idx]}...")
        X_test_text, y_test = load_data(test_file)
        X_test = vectorizer.transform(X_test_text)
        
        y_pred = model.predict(X_test)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        metric_str = f"Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}, Accuracy={accuracy:.3f}"
        row_train.append(metric_str)
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix: {algorithm}\nTrain: Combined Train Test: {test_names[idx]}')
        plt.savefig(f'confusion_matrices/cm_{algorithm.replace(" ", "_")}_TrainCombined_{test_names[idx].replace(" ", "")}.png')
        plt.close()
    table.append(row_train)

# Ispis tablice u markdown formatu
header = "| #      | method           | algorithm                | skup          | Test 1                                                  | Test 2                                                  | Test 3                                                  |"
sep =    "|--------|------------------|--------------------------|--------------|---------------------------------------------------------|---------------------------------------------------------|---------------------------------------------------------|"
print("\n" + header)
print(sep)
for row in table:
    print(f"| {row[0]:<6} | {row[1]:<16} | {row[2]:<24} | {row[3]:<12} | {row[4]:<55} | {row[5]:<55} | {row[6]:<55} |")

# Spremi rezultate u .md datoteku
with open('results_group2.md', 'w', encoding='utf-8') as f:
    f.write(header + "\n")
    f.write(sep + "\n")
    for row in table:
        f.write(f"| {row[0]:<6} | {row[1]:<16} | {row[2]:<24} | {row[3]:<12} | {row[4]:<55} | {row[5]:<55} | {row[6]:<55} |\n")



Treniranje modela: Logistic regression...
Predikcija na Test 1...
Predikcija na Test 2...
Predikcija na Test 3...

Treniranje modela: SVM RBF kernel...
Predikcija na Test 1...
Predikcija na Test 2...
Predikcija na Test 3...

| #      | method           | algorithm                | skup          | Test 1                                                  | Test 2                                                  | Test 3                                                  |
|--------|------------------|--------------------------|--------------|---------------------------------------------------------|---------------------------------------------------------|---------------------------------------------------------|
| 1.a.i  | Machine learning | Logistic regression      | Train combined | Precision=0.640, Recall=0.614, F1=0.625, Accuracy=0.614 | Precision=0.632, Recall=0.630, F1=0.626, Accuracy=0.630 | Precision=0.717, Recall=0.691, F1=0.686, Accuracy=0.691 |
| 1.b.i  | Machine learning | SVM